In [ ]:
param ={
            'num_leaves': [4,5,6,7,8],  
             'max_depth':[-1],
             'learning_rate': [0.1],
             "n_estimators": [100],
             "min_child_samples":[20,40,60,80],
             #'subsample': [0.1], 
             #'colsample_bytree': [1,0.95],
             #'reg_alpha': [1,0,1e-1],
             #'reg_lambda': [10,5,3,1,0,1e-1,1e-2],
            #'random_seed' : [0,1,2,3]
            'boosting':['gbdt']
             }
lgb =LGBMClassifier()
lgb_clf = GridSearchCV(lgb,param,scoring='accuracy',n_jobs=num_thread,cv=5)
lgb_results = lgb_clf.fit(X_train, y_train.values.ravel())

train_score = lgb_results.score(X_train, y_train)
test_score = lgb_results.score(X_test, y_test)
print(lgb_results.best_params_)
print(test_score)

In [ ]:
col_tmp = X_test.columns
X_train=X_train.values
y_train=y_train.values
X_test=X_test.values
y_test=y_test.values
y_test = np.squeeze(y_test)
y_train = np.squeeze(y_train)

In [ ]:
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import ParameterGrid,KFold
# baysian search 
# n_d, n_a, n_steps 적절한 param 찾기
test = []
kf = KFold(n_splits=5, random_state=42, shuffle=True)
preds = []
scores  = []
param_grid = dict(n_d = [2,4,6,8],
                  n_a = [2,4,6,8],
                  n_steps = [2, 3, 4, 5],
                  #lr = [0.1,0.01,],
                  #gamma = [1, 1.5, 2],
                  #lambda_sparse = [1e-1, 1e-2, 1e-3],
                  #n_shared = [2],
                  #n_independent = [2],
                  #clip_value = [2.],     
)
grid = ParameterGrid(param_grid)
max_val_acc = 0
test_acc =0 
best_param = 0
for params in grid:
    scores = []
    preds = []
    for trn_idx,val_idx in kf.split(X_train):
        X_trn = X_train[trn_idx]
        y_trn = y_train[trn_idx]
        X_val = X_train[val_idx]
        y_val = y_train[val_idx]

        clf = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                            optimizer_params=dict(lr=1e-2),
                            scheduler_params={"step_size":50,
                                                "gamma":0.9},
                            scheduler_fn=torch.optim.lr_scheduler.StepLR,
                            mask_type='sparsemax' # "sparsemax", entmax
                            )

        max_epochs = 2

        clf.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_trn, y_trn), (X_val, y_val)],
            eval_name=['train', 'val'],
            eval_metric=['auc','accuracy'],
            max_epochs=max_epochs , patience=10,
            batch_size=128, virtual_batch_size=64,
            num_workers=0,
            weights=1,
            drop_last=True,
        )

        scores.append(clf.best_cost)
        
        pred = clf.predict(X_test)
        preds.append(pred)
    val_acc = np.mean(scores)
    if max_val_acc < val_acc:
        max_val_acc = val_acc
        best_param = params
        test_acc = np.mean(preds)
print('best_param: ', best_param)
print('best_val_acc: ', max_val_acc)
print('test_acc: ', test_acc)

In [ ]:
# 위의 n_d, n_a, n_steps best param 넣고 실행
param_grid = dict(n_d = [],
                  n_a = [],
                  n_steps = [],
                  lr = [0.1,0.01],
                  gamma = [1, 1.5, 2],
                  #lambda_sparse = [1e-1, 1e-2, 1e-3],
                  n_shared = [1,2,3,4],
                  n_independent = [1,2,3,4],  
)
grid = ParameterGrid(param_grid)
max_val_acc = 0
test_acc =0 
best_param = 0
for params in grid:
    scores = []
    preds = []
    for trn_idx,val_idx in kf.split(X_train):
        X_trn = X_train[trn_idx]
        y_trn = y_train[trn_idx]
        X_val = X_train[val_idx]
        y_val = y_train[val_idx]

        clf = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                            optimizer_params=dict(lr=1e-2),
                            scheduler_params={"step_size":50,
                                                "gamma":0.9},
                            scheduler_fn=torch.optim.lr_scheduler.StepLR,
                            mask_type='sparsemax' # "sparsemax", entmax
                            )

        max_epochs = 2

        clf.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_trn, y_trn), (X_val, y_val)],
            eval_name=['train', 'val'],
            eval_metric=['auc','accuracy'],
            max_epochs=max_epochs , patience=10,
            batch_size=128, virtual_batch_size=64,
            num_workers=0,
            weights=1,
            drop_last=True,
        )

        scores.append(clf.best_cost)
        
        pred = clf.predict(X_test)
        preds.append(pred)
    val_acc = np.mean(scores)
    if max_val_acc < val_acc:
        max_val_acc = val_acc
        best_param = params
        test_acc = np.mean(preds)
print('best_param: ', best_param)
print('best_val_acc: ', max_val_acc)
print('test_acc: ', test_acc)